In [5]:
# TODO: remove cell
from articlecommon import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
lgbm = LGBMClassifier(random_state=42, verbose=-1)
psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")
X_le_psc = pd.concat([X_le, psc_ohe], axis=1)



psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")

counts = X_le["Policy_Sales_Channel"].value_counts()

counts = counts[counts == 1]
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(x), counts.index.tolist()))
psc_ohe = psc_ohe.drop(columns=columns_to_remove)

X_le_psc = pd.concat([X_le, psc_ohe], axis=1)

value_counts = X_le_psc["Policy_Sales_Channel"].value_counts().index
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(value_counts[x]), [*range(147, 86, -1)]))
columns_to_remove += ['Vintage']

from category_encoders import TargetEncoder
X_le_psc_te = X_le_psc.drop(columns=columns_to_remove).copy()
X_le_psc_te['Policy_Sales_Channel_te'] = X_le_psc_te['Policy_Sales_Channel']

X_le_psc_te_rc = X_le_psc_te.copy()
X_le_psc_te_rc['Region_Code_te'] = X_le_psc_te_rc['Region_Code']
selected_to_remove = pd.read_csv('results/columns_to_remov4.csv', index_col=0)["0"]

X_final = X_le_psc_te_rc
X_rc_ohe = X_final.drop(columns=selected_to_remove)
rc_ohe = pd.get_dummies(X_rc_ohe["Region_Code"], prefix="Region_Code")
X_rc_ohe = pd.concat([X_rc_ohe, rc_ohe], axis=1)

fixed_columns = [x for x in X_rc_ohe if x not in rc_ohe.columns]
columns_to_add = ["Region_Code_0.0", "Region_Code_1.0", "Region_Code_38.0", "Region_Code_52.0"]

encoder = TargetEncoder(cols=['Region_Code', 'Policy_Sales_Channel'])
# X_reduce_te = X_rc_ohe[fixed_columns + columns_to_add]
X_reduce_te = X_rc_ohe.loc[:, fixed_columns + columns_to_add]

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]



In [6]:
import optuna
np.seterr(all="ignore")

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]
transformer = ColumnTransformer(transformers=[('pwrtrans', PowerTransformer(), all_columns)])


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1.0)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    subsample = trial.suggest_uniform('subsample', 0.4, 1.0)
    te_smoothing = trial.suggest_loguniform("te_smoothing", 0.05, 10.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7)
    gamma = trial.suggest_uniform("gamma", 0.0, 1.0)
    colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.3, 1.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-3, 1e2)

    
    xgb = XGBClassifier(random_state=42, eval_metric="logloss", learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, subsample=subsample, min_child_weight=min_child_weight, gamma=gamma, colsample_bytree=colsample_bytree, reg_alpha=reg_alpha)
    encoder = TargetEncoder(cols=['Policy_Sales_Channel', 'Region_Code'], smoothing=te_smoothing)
    model = make_pipeline(encoder, transformer, xgb)

    score = cross_val_score(model, X_reduce_te, y, cv=10, scoring='roc_auc', n_jobs=-1).mean() # TODO: set cv=10 and the whole dataset
    print("  ROC AUC: {}".format(score))

    print("  Params for trial " + str(trial.number) + ": ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return score

In [13]:
trials_number = 1 # TODO: set higer value

study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage_name)

study.enqueue_trial({
        "learning_rate": 0.05014352877833517,
        "max_depth": 7,
        "min_child_weight": 5,
        "n_estimators": 234,
        "subsample": 0.9009423215546716,
        "class_weight": None,
        "te_smoothing": 0.20145207364872467,
        "gamma": 0.2491618356357863,
        "colsample_bytree": 0.631803937308591,
        "reg_alpha": 0.025408324931767336
    })

optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=2)

print("\nBest trial:")
trial = study.best_trial

print("  ROC AUC: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  ROC AUC: 0.8590493522316688
  Params for trial 0: 
    learning_rate: 0.05014352877833517
    max_depth: 7
    n_estimators: 234
    subsample: 0.9009423215546716
    te_smoothing: 0.20145207364872467
    min_child_weight: 5
    gamma: 0.2491618356357863
    colsample_bytree: 0.631803937308591
    reg_alpha: 0.025408324931767336
  ROC AUC: 0.8365101583894594
  Params for trial 1: 
    learning_rate: 0.0011408547793103705
    max_depth: 2
    n_estimators: 87
    subsample: 0.872274827571086
    te_smoothing: 2.7737921694052
    min_child_weight: 5
    gamma: 0.8941621038661537
    colsample_bytree: 0.5936983836843396
    reg_alpha: 2.7880713148493594

Best trial:
  ROC AUC: 0.8590493522316688
  Params: 
    colsample_bytree: 0.631803937308591
    gamma: 0.2491618356357863
    learning_rate: 0.05014352877833517
    max_depth: 7
    min_child_weight: 5
    n_estimators: 234
    reg_alpha: 0.025408324931767336
    subsample: 0.9009423215546716
    te_smoothing: 0.20145207364872467


In [15]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8548510536519618
  Params for trial 2: 
    learning_rate: 1.171447845733699e-06
    max_depth: 7
    n_estimators: 151
    subsample: 0.5956965122416562
    te_smoothing: 0.12938532073884987
    min_child_weight: 1
    gamma: 0.0614557455025182
    colsample_bytree: 0.5678484025475484
    reg_alpha: 0.004983703091142559
  ROC AUC: 0.8483427679078958
  Params for trial 3: 
    learning_rate: 0.41064032199860423
    max_depth: 6
    n_estimators: 348
    subsample: 0.6487171329341139
    te_smoothing: 6.066659988561655
    min_child_weight: 2
    gamma: 0.6543065471891478
    colsample_bytree: 0.6299164530369348
    reg_alpha: 0.02148369026716384
  ROC AUC: 0.8491445513844462
  Params for trial 4: 
    learning_rate: 0.00025625592238240255
    max_depth: 10
    n_estimators: 321
    subsample: 0.5492090723913845
    te_smoothing: 5.867968850337677
    min_child_weight: 4
    gamma: 0.21501915035894026
    colsample_bytree: 0.36560040996361715
    reg_alpha: 0.10533087307543

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

  ROC AUC: 0.8516931862087664
  Params for trial 9: 
    learning_rate: 9.294535824216191e-05
    max_depth: 8
    n_estimators: 207
    subsample: 0.7751932635294727
    te_smoothing: 0.6237764623674533
    min_child_weight: 6
    gamma: 0.7172766596492102
    colsample_bytree: 0.4108837527246837
    reg_alpha: 2.765207229513734


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8559324238631698
  Params for trial 10: 
    learning_rate: 0.016290287896287295
    max_depth: 4
    n_estimators: 409
    subsample: 0.7762381901211387
    te_smoothing: 0.057371721726448596
    min_child_weight: 7
    gamma: 0.36504771836541167
    colsample_bytree: 0.4876975301312798
    reg_alpha: 67.05447303767998
  ROC AUC: 0.8589972062530837
  Params for trial 11: 
    learning_rate: 0.036542272962765805
    max_depth: 5
    n_estimators: 488
    subsample: 0.9872363299520472
    te_smoothing: 0.3522600407665465
    min_child_weight: 5
    gamma: 0.9592782847620096
    colsample_bytree: 0.7742444553853193
    reg_alpha: 1.0712508073655056
  ROC AUC: 0.8570262774240598
  Params for trial 12: 
    learning_rate: 0.018587552872626187
    max_depth: 4
    n_estimators: 486
    subsample: 0.9993580513744696
    te_smoothing: 0.48985712619598226
    min_child_weight: 5
    gamma: 0.5343990580432245
    colsample_bytree: 0.7775235309535705
    reg_alpha: 0.743176932407509

KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8535110287214007
  Params for trial 15: 
    learning_rate: 0.003993462052981665
    max_depth: 5
    n_estimators: 173
    subsample: 0.8953349792182
    te_smoothing: 1.2269721773360744
    min_child_weight: 3
    gamma: 0.9990771853679652
    colsample_bytree: 0.6933005653662576
    reg_alpha: 0.29610458208646356
  ROC AUC: 0.858677144562259
  Params for trial 16: 
    learning_rate: 0.05866692581456225
    max_depth: 7
    n_estimators: 279
    subsample: 0.7739614962045015
    te_smoothing: 0.06929382239644655
    min_child_weight: 6
    gamma: 0.5148763179045638
    colsample_bytree: 0.985102117876606
    reg_alpha: 0.27869508027647494
  ROC AUC: 0.8549251015043341
  Params for trial 17: 
    learning_rate: 0.004850909035816609
    max_depth: 5
    n_estimators: 430
    subsample: 0.940482963838627
    te_smoothing: 0.23943443847743945
    min_child_weight: 3
    gamma: 0.002001439919436576
    colsample_bytree: 0.5155018578128264
    reg_alpha: 1.454163994834919


KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8536102995348649
  Params for trial 19: 
    learning_rate: 2.0431499169614177e-05
    max_depth: 8
    n_estimators: 341
    subsample: 0.8265996971396086
    te_smoothing: 0.10251459922537497
    min_child_weight: 5
    gamma: 0.3669671029305389
    colsample_bytree: 0.8193060606353695
    reg_alpha: 18.534383975262863
  ROC AUC: 0.8495505727309638
  Params for trial 20: 
    learning_rate: 0.9982728923934819
    max_depth: 4
    n_estimators: 211
    subsample: 0.6781628535946181
    te_smoothing: 0.9899365930414389
    min_child_weight: 4
    gamma: 0.821821991177983
    colsample_bytree: 0.7097387992355282
    reg_alpha: 0.03978602985400263
  ROC AUC: 0.8540198308132435
  Params for trial 21: 
    learning_rate: 0.003760587990643622
    max_depth: 6
    n_estimators: 158
    subsample: 0.9439195897171943
    te_smoothing: 0.4032490998866837
    min_child_weight: 6
    gamma: 0.21461438986900438
    colsample_bytree: 0.8567799045090675
    reg_alpha: 0.1194808078927235

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

  ROC AUC: 0.8575716804895134
  Params for trial 23: 
    learning_rate: 0.04316794123198071
    max_depth: 10
    n_estimators: 408
    subsample: 0.8894149181767028
    te_smoothing: 0.3359623254509779
    min_child_weight: 6
    gamma: 0.24168093118581563
    colsample_bytree: 0.7468877056823744
    reg_alpha: 0.0013401704517960382


KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8587971122312842
  Params for trial 25: 
    learning_rate: 0.0125138037147974
    max_depth: 9
    n_estimators: 437
    subsample: 0.8402511867387887
    te_smoothing: 0.6908435817917737
    min_child_weight: 5
    gamma: 0.30385989418742193
    colsample_bytree: 0.8105551139946813
    reg_alpha: 0.001528221804249006
  ROC AUC: 0.8539796268027396
  Params for trial 26: 
    learning_rate: 0.13826315454794816
    max_depth: 9
    n_estimators: 390
    subsample: 0.9271471278219922
    te_smoothing: 0.11562440897569526
    min_child_weight: 6
    gamma: 0.442895596678844
    colsample_bytree: 0.6866263312663199
    reg_alpha: 0.01616903577118913
  ROC AUC: 0.8582696060856929
  Params for trial 27: 
    learning_rate: 0.047330908013621775
    max_depth: 8
    n_estimators: 372
    subsample: 0.8349754728896656
    te_smoothing: 0.26148654681994077
    min_child_weight: 5
    gamma: 0.15045655634307142
    colsample_bytree: 0.9047410971255911
    reg_alpha: 0.003054937754978

KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8578492502398347
  Params for trial 31: 
    learning_rate: 0.008569721104088672
    max_depth: 7
    n_estimators: 455
    subsample: 0.9545072259526552
    te_smoothing: 3.311999689266554
    min_child_weight: 4
    gamma: 0.3057017361604384
    colsample_bytree: 0.44885621996377223
    reg_alpha: 0.846453716434768


KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8584196646622544
  Params for trial 37: 
    learning_rate: 0.030341501856637865
    max_depth: 9
    n_estimators: 499
    subsample: 0.875821212514766
    te_smoothing: 0.4926922397568707
    min_child_weight: 6
    gamma: 0.2784498896778679
    colsample_bytree: 0.7286505061365376
    reg_alpha: 0.010444247910627357
  ROC AUC: 0.8558705849765603
  Params for trial 38: 
    learning_rate: 0.002079716741465758
    max_depth: 9
    n_estimators: 456
    subsample: 0.9099632020157327
    te_smoothing: 0.1941680350601603
    min_child_weight: 5
    gamma: 0.16927299069338286
    colsample_bytree: 0.5495137798306925
    reg_alpha: 1.9163562239046628
  ROC AUC: 0.8575467256166313
  Params for trial 39: 
    learning_rate: 0.0077962613913499556
    max_depth: 7
    n_estimators: 364
    subsample: 0.9938537697878643
    te_smoothing: 0.13497451206888486
    min_child_weight: 5
    gamma: 0.07962653638814793
    colsample_bytree: 0.6406077968843318
    reg_alpha: 0.0027956928820

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

  ROC AUC: 0.8581755605139619
  Params for trial 44: 
    learning_rate: 0.008855186621272717
    max_depth: 7
    n_estimators: 462
    subsample: 0.6634463551757472
    te_smoothing: 8.824974633831767
    min_child_weight: 1
    gamma: 0.01741105669215748
    colsample_bytree: 0.5519115887543354
    reg_alpha: 0.5279955718167861


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8577530937588413
  Params for trial 45: 
    learning_rate: 0.20624790440672675
    max_depth: 5
    n_estimators: 252
    subsample: 0.5696276265662975
    te_smoothing: 0.28701680562668996
    min_child_weight: 2
    gamma: 0.1871774181965124
    colsample_bytree: 0.6071046006637101
    reg_alpha: 4.998799232950193
  ROC AUC: 0.8560286567867473
  Params for trial 46: 
    learning_rate: 0.0006165356621419122
    max_depth: 8
    n_estimators: 193
    subsample: 0.7437159968904659
    te_smoothing: 0.9470304664789755
    min_child_weight: 2
    gamma: 0.11516294035844335
    colsample_bytree: 0.7118285437030619
    reg_alpha: 0.025417032378898308
  ROC AUC: 0.8584270885058108
  Params for trial 47: 
    learning_rate: 0.031054227727161102
    max_depth: 9
    n_estimators: 415
    subsample: 0.964342464870707
    te_smoothing: 0.5984972155570518
    min_child_weight: 6
    gamma: 0.26597870188278083
    colsample_bytree: 0.8473770560063929
    reg_alpha: 0.088798330684947

KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8590739770495592
  Params for trial 49: 
    learning_rate: 0.021291366221284337
    max_depth: 8
    n_estimators: 476
    subsample: 0.8706268352745364
    te_smoothing: 0.35648520716557985
    min_child_weight: 5
    gamma: 0.20241066830358567
    colsample_bytree: 0.759012193204171
    reg_alpha: 0.010982331080557824


KeyboardInterrupt: 

In [7]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8587770986761969
  Params for trial 51: 
    learning_rate: 0.020297763024509443
    max_depth: 6
    n_estimators: 479
    subsample: 0.9279269975125053
    te_smoothing: 0.21622487613437308
    min_child_weight: 1
    gamma: 0.074134315948431
    colsample_bytree: 0.7368408384708678
    reg_alpha: 0.010415402027620292
  ROC AUC: 0.8566035313229314
  Params for trial 52: 
    learning_rate: 0.0027843315512583134
    max_depth: 7
    n_estimators: 453
    subsample: 0.5260837570342178
    te_smoothing: 0.16813094979345272
    min_child_weight: 4
    gamma: 0.21460658019302825
    colsample_bytree: 0.661406845155227
    reg_alpha: 0.031507177930789075
  ROC AUC: 0.8573861408932961
  Params for trial 53: 
    learning_rate: 0.061091523214698656
    max_depth: 8
    n_estimators: 498
    subsample: 0.8095631914648187
    te_smoothing: 0.4464050042575537
    min_child_weight: 5
    gamma: 0.6894922593442481
    colsample_bytree: 0.6869135331288123
    reg_alpha: 0.162980912798

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8587619010345605
  Params for trial 58: 
    learning_rate: 0.0944945310745452
    max_depth: 4
    n_estimators: 332
    subsample: 0.7641353432703742
    te_smoothing: 0.5620805567780408
    min_child_weight: 3
    gamma: 0.11564880908550024
    colsample_bytree: 0.7186169020544344
    reg_alpha: 0.061663871312557725


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

...

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8587475486842179
  Params for trial 67: 
    learning_rate: 0.043915473162132894
    max_depth: 6
    n_estimators: 222
    subsample: 0.8475514901379255
    te_smoothing: 0.7379541250486278
    min_child_weight: 5
    gamma: 0.15919586040932981
    colsample_bytree: 0.8254696335259345
    reg_alpha: 0.0010078813947508496
  ROC AUC: 0.855508770427862
  Params for trial 68: 
    learning_rate: 0.0056842853366176884
    max_depth: 8
    n_estimators: 360
    subsample: 0.9324910751962884
    te_smoothing: 0.14561974348321827
    min_child_weight: 6
    gamma: 0.26799009623535136
    colsample_bytree: 0.6717331639439187
    reg_alpha: 46.75487398839003
  ROC AUC: 0.858183085370517
  Params for trial 69: 
    learning_rate: 0.013724417597489803
    max_depth: 8
    n_estimators: 267
    subsample: 0.9889197372619398
    te_smoothing: 0.2887625288181512
    min_child_weight: 4
    gamma: 0.292717963157657
    colsample_bytree: 0.7892796630188916
    reg_alpha: 0.260335516089459

KeyboardInterrupt: 